In [1]:
import numpy as np
import tensorflow as tf
import os

In [8]:
def generate_linear_data(dimension, num):
    """
    随机产生线性模型数据
    参数
    ----
    dimension ：int，自变量个数
    num ：int，数据个数
    返回
    ----
    x ：np.array，自变量
    y ：np.array，因变量
    """
    np.random.seed(1024)
    beta = np.array(range(dimension)) + 1
    x = np.random.random((num, dimension))
    epsilon = np.random.random((num, 1))
    # 将被预测值写成矩阵形式，会极大加快速度
    y = x.dot(beta).reshape((-1, 1)) + epsilon
    return x, y

In [9]:
def create_linear_model(dimension):
    """
    搭建模型，包括数据中的自变量，应变量和损失函数
    参数
    ----
    dimension : int，自变量的个数
    返回
    ----
    model ：dict，里面包含模型的参数，损失函数，自变量，应变量
    """
    np.random.seed(1024)
    # 定义自变量和应变量
    x = tf.placeholder(tf.float64, shape=[None, dimension], name='x')
    # 将被预测值写成矩阵形式，会极大加快速度
    y = tf.placeholder(tf.float64, shape=[None, 1], name="y")
    # 定义参数估计值和预测值
    beta_pred = tf.Variable(np.random.random([dimension, 1]))
    y_pred = tf.matmul(x, beta_pred, name="y_pred")
    # 定义损失函数
    loss = tf.reduce_mean(tf.square(y_pred - y))
    model = {"loss_function": loss, "independent_variable": x,
             "dependent_variable": y, "model_params": beta_pred, "prediction": y_pred, }
    return model

In [10]:
def gradient_descent(X, Y, model, learning_rate=0.01, max_iter=10000, tol=1.e-6):
    """
    利用梯度下降法训练模型。
    参数
    ----
    X : np.array, 自变量数据
    Y : np.array, 因变量数据
    model : dict, 里面包含模型的参数，损失函数，自变量，应变量。
    """
    # 确定最优化算法
    method = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = method.minimize(model["loss_function"])
    # 增加日志
    tf.summary.scalar("loss_function", model["loss_function"])
    tf.summary.histogram("params", model["model_params"])
    tf.summary.scalar("first_param", tf.reduce_mean(model["model_params"][0]))
    tf.summary.scalar("last_param", tf.reduce_mean(model["model_params"][-1]))
    summary = tf.summary.merge_all()
    # 在程序运行结束之后，运行如下命令，查看日志
    # tensorboard --logdir logs/
    # Windows下的存储路径与Linux并不相同
    if os.name == "nt":
        summary_writer = create_summary_writer("logs\\gradient_descent")
    else:
        summary_writer = create_summary_writer("logs/gradient_descent")
    # tensorflow开始运行
    sess = tf.Session()
    # 产生初始参数
    init = tf.global_variables_initializer()
    # 用之前产生的初始参数初始化模型
    sess.run(init)
    # 迭代梯度下降法
    step = 0
    prev_loss = np.inf
    diff = np.inf
    # 当损失函数的变动小于阈值或达到最大循环次数，则停止迭代
    while (step < max_iter) & (diff > tol):
        _, summary_str, loss = sess.run(
            [optimizer, summary, model["loss_function"]],
            feed_dict={model["independent_variable"]: X,
                       model["dependent_variable"]: Y})
        # 将运行细节写入目录
        summary_writer.add_summary(summary_str, step)
        # 计算损失函数的变动
        diff = abs(prev_loss - loss)
        prev_loss = loss
        step += 1
    summary_writer.close()
    # 在Windows下运行此脚本需确保Windows下的命令提示符(cmd)能显示中文
    # 输出最终结果
    print("模型参数：\n%s" % sess.run(model["model_params"]))
    print("迭代次数：%s" % step)
    print("损失函数值：%s" % loss)
    sess.close()

In [11]:
def create_summary_writer(log_path):
    """
    检查所给路径是否已存在，如果存在删除原有日志。并创建日志写入对象
    参数
    ----
    logPath ：string，日志存储路径
    返回
    ----
    summaryWriter ：FileWriter，日志写入器
    """
    if tf.gfile.Exists(log_path):
        tf.gfile.DeleteRecursively(log_path)
    summary_writer = tf.summary.FileWriter(log_path, graph=tf.get_default_graph())
    return summary_writer

In [12]:
# dimension表示自变量的个数，num表示数据集里数据的个数。
dimension = 20
num = 10000
# 重置tensorflow
tf.reset_default_graph()
# 随机产生模型数据
X, Y = generate_linear_data(dimension, num)
# 定义模型
model = create_linear_model(dimension)
print(model["independent_variable"])
print(model["dependent_variable"])
# 使用梯度下降法，估计模型参数
gradient_descent(X, Y, model)

Tensor("x:0", shape=(?, 20), dtype=float64)
Tensor("y:0", shape=(?, 1), dtype=float64)
模型参数：
[[ 1.07543411]
 [ 2.07516432]
 [ 3.05854161]
 [ 4.05751402]
 [ 5.05116987]
 [ 6.07425363]
 [ 7.05268754]
 [ 8.04987389]
 [ 9.07500358]
 [10.05527314]
 [11.03118603]
 [12.05141335]
 [13.04937497]
 [14.03702336]
 [15.02553041]
 [16.0384729 ]
 [17.02755372]
 [18.04220439]
 [19.03484184]
 [20.02479193]]
迭代次数：3654
损失函数值：0.08738502526303255
